In [ ]:
def get_connected(market, trade_type):
    ''' get connected to ibkr
    Args: 
       (market) as string <'nse'> | <'snp'>
       (trade_type) as string <'live'> | <'paper'>
    Returns:
        (ib) object if successful
    '''
    
    ip = (market.upper(), trade_type.upper())
    
    #host dictionary
    hostdict = {('NSE', 'LIVE'): 3000,
                ('NSE', 'PAPER'): 3001,
                ('SNP', 'LIVE'): 1300,
                ('SNP', 'PAPER'): 1301,}
    
    host = hostdict[ip]
    
    cid = 1 # initialize clientId
    max_cid = 5 # maximum clientId allowed. max possible is 32

    for i in range(cid, max_cid):
        try:
            ib = IB().connect('127.0.0.1', host, clientId=i)
            
        except Exception as e:
            print(e) # print the error
            continue # go to next
            
        break # successful try
        
    return ib

In [ ]:
from itertools import repeat
def get_margins(ib, contracts, *lotsize):
    '''Margin dictionary. 1 min for 100 contracts.
    Args:
        (ib) as object
        (contracts) as list of underlying contract
        (*lotsize) as <int>|<list>
    Returns:
        {contract (obj): underlying_margin(float)} as dictionary'''
    
    if type(contracts) is pd.Series:
        contracts = list(contracts)
    else:
        contracts = contracts

    if type(lotsize[0]) is pd.Series:
        positions = list(lotsize[0])
    else:
        positions = repeat(lotsize[0], len(contracts)) # convert *arg tuple to int
    
    orders = [Order(action='SELL', orderType='MKT', totalQuantity=abs(p), whatIf=True)
              if p < 0 else
              Order(action='BUY', orderType='MKT', totalQuantity=abs(p), whatIf=True)
              for p in positions]

    co = [c for c in zip(contracts, orders)]

    dict_margins = {c: float(ib.whatIfOrder(c, o).initMarginChange) for c, o in co}
    
    return dict_margins

In [ ]:
import pandas as pd
def get_underlyings(ib):
    '''Returns: list of underlying contracts
    Usage: 
       with get_connected('snp', 'live') as ib: und_contracts = get_underlyings(ib)'''

    # exclusion list
    excl = ['VXX','P', 'TSRO']

    # Download cboe weeklies to a dataframe
    dls = "http://www.cboe.com/publish/weelkysmf/weeklysmf.xls"

#     snp500 = list(pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0][1:].loc[:, 1])

    snp100 = pd.read_html('https://en.wikipedia.org/wiki/S%26P_100')[2][1:].loc[:, 0]
    snp100 = [s.replace('.', ' ') if '.' in s else s  for s in snp100] # without dot in symbol
    # read from row no 11, dropna and reset index
    df_cboe = pd.read_excel(dls, header=12, 
                            usecols=[0,2,3]).loc[11:, :]\
                            .dropna(axis=0)\
                            .reset_index(drop=True)

    # remove column names white-spaces and remap to IBKR
    df_cboe.columns = df_cboe.columns.str.replace(' ', '')

    # remove '/' for IBKR
    df_cboe.Ticker = df_cboe.Ticker.str.replace('/', ' ', regex=False)

    # make symbols
    symbols = {s for s in df_cboe.Ticker if s not in excl if s in snp100}
    stocks = [Stock(symbol=s, exchange='SMART', currency='USD') for s in symbols]

    blk = 50 # no of stocks in a block
    stkblks = [stocks[i: i+blk] for i in range(0, len(stocks), blk)] # blocks of stocks

    # qualify the contracts
    contracts = [ib.qualifyContracts(*s) for s in stkblks]

    # return flattened contract list
    return [contract for subl in contracts for contract in subl] 

In [ ]:
# gets days to expiry from now onwards
import datetime

def get_dte(dt):
    '''Gets days to expiry
    Arg: (dt) as day in string format 'yyyymmdd'
    Returns: days to expiry as int'''
    return (util.parseIBDatetime(dt) - 
            datetime.datetime.now().date()).days

In [ ]:
from math import sqrt
def get_rollingmax_std(c, dte, durmult=3):
    '''gets the rolling max standard deviation
    Args:
        (c) as contract object
        (dte) as int for no of days for expiry
        (durmult) no of samples to go backwards on
    Returns:
        maximum rolling standard deviation as int'''

    durStr = str(durmult*dte) + ' D' # Duration String
    tradingdays = 252
    
    # Extract the history
    hist = ib.reqHistoricalData(contract=c, endDateTime='', 
                                    durationStr=durStr, barSizeSetting='1 day',  
                                                whatToShow='Trades', useRTH=True)
    df = util.df(hist)
    df.insert(0, column='symbol', value=c.symbol)

    df_ohlc = df.set_index('date').sort_index(ascending = False)

    # get cumulative standard deviation
    df_stdev = pd.DataFrame(df_ohlc['close'].expanding(1).std(ddof=0))
    df_stdev.columns = ['stdev']

    # get cumulative volatility
    df_vol = pd.DataFrame(df_ohlc['close'].pct_change().expanding(1).std(ddof=0)*sqrt(tradingdays))
    df_vol.columns = ['volatility']

    df_ohlc1 = df_ohlc.join(df_vol)

    df_ohlc2 = df_ohlc1.join(df_stdev)

    return df_stdev.stdev.max()

In [ ]:
def get_maxfallrise(c, dte):
    '''get the maximum rise and fall for rolling window of dte
    Args:
       (c) as the underlying contract object
       (dte) as int for days to expiry of a contract
    Returns:
       (max_fall, max_rise) tuple of floats'''
    
    
    hist = ib.reqHistoricalData(contract=c, endDateTime='', 
                                        durationStr='365 D', barSizeSetting='1 day',  
                                                    whatToShow='Trades', useRTH=True)

    df = util.df(hist)
    df.insert(0, column='symbol', value=c.symbol)

    df_ohlc = df.set_index('date').sort_index(ascending=True)
    df = df_ohlc.assign(delta=df_ohlc.high.rolling(dte).max()-df_ohlc.low.rolling(dte).min(), pctchange=df_ohlc.high.pct_change(periods=dte))

    df1 = df.sort_index(ascending=False)
    max_rise = df1[df1.pctchange<=0].delta.max()
    max_fall = df1[df1.pctchange>0].delta.max()
    
    return(max_rise, max_fall)

In [ ]:
# error catching for list comprehension
def catch(func, handle=lambda e : e, *args, **kwargs):
    '''List comprehension error catcher
    Args: 
        (func) as the function
         (handle) as the lambda of function
         <*args | *kwargs> as arguments to the functions
    Outputs:
        output of the function | <np.nan> on error
    Usage:
        eggs = [1,3,0,3,2]
        [catch(lambda: 1/egg) for egg in eggs]'''
    try:
        return func(*args, **kwargs)
    except Exception as e:
        return np.nan

In [ ]:
from ib_insync import *
util.startLoop()

# get connected
try:
    ib.isConnected
except Exception as e:
    ib = get_connected('snp', 'live')

In [ ]:
%%time
#____________________________________________________________________________________

# This section needs to be a script.
# It needs to be run once first to get df_p and remaining quantities

import numpy as np

if not ib.isConnected():
    ib = get_connected('snp', 'live')

m_maxp = 0.02     # % of max margin allowed on net liquidity per scrip to limit positon risk
max_nlvp = 0.8    # max allowable nlv to prevent overall portfolio risk. 0.8 means 80% of NLV.
                  # max available funds for option trades = max_nlvp * NLV - initMargin

maxdte = 60       # maximum days-to-expiry for options
minstdmult = 3    # minimum standard deviation multiple to screen strikes. 3 is 99.73% probability
    
fspath = '../data/snp/' # path for pickles
       
#...get current positions
#________________________

#... read the account info
ac = ib.accountValues()
df_a = util.df(ac)

#... set max margin per position
net_liq = float(df_a[df_a.tag == 'NetLiquidation'].iloc[0].value) 
# av_funds = float(df_a[df_a.tag == 'FullAvailableFunds'].iloc[0].value)
max_p = net_liq*m_maxp

#... read the positions
ps = ib.portfolio()
df_p = util.df(ps)

expiry = [d.lastTradeDateOrContractMonth for d in df_p.contract]
strike = [d.strike for d in df_p.contract]

df_p['symbol'] = [s.symbol for s in df_p.contract.values]
df_p['expiry'] = expiry
df_p['strike'] = strike

df_p = df_p.assign(type=[type(x).__name__ for x in df_p.contract])

# get the contracts again (as some of them miss markets)
port_c = [j for c in [ib.qualifyContracts(Contract(conId=c.conId)) for c in df_p.contract] for j in c]

df_p = df_p.assign(contract=port_c)

# get the margins
dict_port_opt_margins = get_margins(ib, df_p.contract, df_p.position)
df_p = df_p.assign(totmargin=df_p.contract.map(dict_port_opt_margins))

df_p = df_p.assign(lotsize=[int(k.multiplier) for k in port_c])

df_p_margin = pd.DataFrame(df_p.groupby('symbol').sum().totmargin).reset_index()

#... get the underlying margins and the blacklist
#____________________________________________________

undContracts = get_underlyings(ib)
lotsize=100
undMargins = get_margins(ib, undContracts, 100)
undMargins = {k.symbol: v for k, v in undMargins.items()} # margin with symbols instead of contracts
undMargins = {k: abs(v) for k, v in undMargins.items()} # positive margins

df_p_margin = df_p_margin.assign(margin = df_p_margin.symbol.map(undMargins))
df_p_margin = df_p_margin.assign(maxmargin=max_p)
df_p_margin = df_p_margin.assign(netqty=((df_p_margin.maxmargin-df_p_margin.totmargin)/df_p_margin.margin).apply(np.floor))
df_p_margin = df_p_margin.assign(remqty=np.where(df_p_margin.netqty > 0, df_p_margin.netqty, 0).astype(int))

blacklist = list(df_p_margin[df_p_margin.remqty <= 0 ].symbol)

#...get the max remaining quantities for each symbol
masterContracts = [c for c in undContracts if c.symbol not in blacklist]
iDict = {c.conId: c for c in masterContracts} # {conId: Contract}
cDict = {v:k for k, v in iDict.items()}       # {Contract: conId}
masterMargin = {m.symbol: undMargins[m.symbol] for m in masterContracts}

maxqty = {k: int(max_p/m) for k, m in masterMargin.items()}
remq = list(df_p_margin[['symbol', 'remqty']].set_index('symbol').to_dict().values())[0]
maxremqty = {k: remq.get(k, v) for k, v in maxqty.items()}

#...get the underlying prices
tickers = ib.reqTickers(*masterContracts)
undPrices = {t.contract.symbol: t.marketPrice() for t in tickers}

#...pick up one master contract and run the code

#****!!! DATA LIMITER **********
undContract = masterContracts[20]
symbol = undContract.symbol

from itertools import product, repeat
chains = ib.reqSecDefOptParams(underlyingSymbol = symbol,
                     futFopExchange = '',
                     underlyingSecType = undContract.secType,
                     underlyingConId= undContract.conId)

xs = [set(product(c.expirations, c.strikes)) for c in chains if c.exchange == 'SMART']

expirations = [i[0] for j in xs for i in j]
strikes = [i[1] for j in xs for i in j]
dflength = len(expirations)

#...first df with symbol, strike and expiry
df1 = pd.DataFrame({'cid': pd.Series(np.repeat(cDict[undContract],dflength)), 
              'symbol': pd.Series(np.repeat(symbol,dflength)),
              'expiration': expirations,
              'strike': strikes,
              'dte': [get_dte(e) for e in expirations],
               'undPrice': pd.Series(np.repeat(undPrices[symbol],dflength))})

df2 = df1[df1.dte < maxdte].reset_index(drop=True)  # limiting dtes

dtes = df2.dte.unique().tolist()

#...get the max fall / rise for puts / calls
maxFallRise = {d: get_maxfallrise(c, d) for c, d in zip(repeat(undContract), dtes)}

df3 = df2.join(pd.DataFrame(df2.dte.map(maxFallRise).tolist(), index=df2.index, columns=['Fall', 'Rise']))

df4 = df3.assign(hiRise = df3.undPrice+df3.Rise, loFall = df3.undPrice-df3.Fall)

std = {d: get_rollingmax_std(c, d) for c, d in zip(repeat(undContract), dtes)}

df4['std3'] = df4.dte.map(std)*3

df4['loStd3'] = df4.undPrice - df4.std3
df4['hiStd3'] = df4.undPrice + df4.std3

# flter puts and calls by standard deviation
df_puts = df4[df4.strike < df4.loStd3]
df_calls = df4[df4.strike > df4.hiStd3]

# df_puts = df4 # keep the puts dataframe without limits
# df_calls = df4.iloc[0:0] # empty the calls dataframe

# with rights
df_puts = df_puts.assign(right='P')
df_calls = df_calls.assign(right='C')

# qualify the options
df_opt1 = pd.concat([df_puts, df_calls]).reset_index()

optipl = [Option(s, e, k, r, 'SMART') for s, e, k, r in zip(df_opt1.symbol, df_opt1.expiration, df_opt1.strike, df_opt1.right)]

q_opt = ib.qualifyContracts(*optipl)

opt_iDict = {c.conId: c for c in q_opt}

df_opt1 = util.df(q_opt).loc[:, ['conId', 'symbol', 'lastTradeDateOrContractMonth', 'strike', 'right']]

df_opt1 = df_opt1.rename(columns={'lastTradeDateOrContractMonth': 'expiration', 'conId': 'optId'})

opt_tickers = ib.reqTickers(*q_opt)

df_opt1 = df_opt1.assign(optPrice = [t.marketPrice() for t in opt_tickers])

df_opt1 = df_opt1[df_opt1.optPrice > 0.0]

In [ ]:
df_calls.head()

In [ ]:
df_opt1